# keras with image MNIST

## pylibs

In [ ]:
import os
import sys
import cv2
import keras
import random
import datetime
import numpy as np
import pandas as pd
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.models import load_model
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from keras.models import Model
from keras import Sequential
from random import randint, choice
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.utils import to_categorical
from keras.callbacks import Callback
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, AveragePooling2D, MaxPooling1D, Conv1D, Reshape

## Extra

In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#more info on https://github.com/Aydinhamedi/Python-color-print/tree/main
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
def print_Color(Input: str, colors: list, print_END: str = '\n', advanced_mode: bool = False):
    """
    Prints colored text to the console using advanced terminal colors.

    Args:
        Input (str): The input string to be printed. In advanced mode, '~*' is used to separate different parts of the string to be printed in different colors.
        colors (list): A list of colors for the text. In non-advanced mode, only the first color in the list is used. In advanced mode, each color corresponds to a part of the input string separated by '~*'.
        print_END (str): The string appended after the final output. Default is '\\n'.
        advanced_mode (bool): If True, enables advanced mode that allows multiple colors in one string. Default is False.

    Examples:
    ~~~python
        print_Color('Hello, World!', ['green']) 
        # Prints 'Hello, World!' in green.

        print_Color('~*Hello in green~*Hello in red', ['green', 'red'], advanced_mode=True) 
        # Prints 'Hello in green' in green and 'Hello in red' in red.

    Note:
        The advanced terminal colors can be used by providing the escape sequences directly in the colors list.
        If an invalid color is provided, an error message will be printed.
    """
    color_code = {
        'black': '\x1b[0;30m',
        'red': '\x1b[0;31m',
        'green': '\x1b[0;32m',
        'yellow': '\x1b[0;33m',
        'blue': '\x1b[0;34m',
        'magenta': '\x1b[0;35m',
        'cyan': '\x1b[0;36m',
        'white': '\x1b[0;37m',
        'normal': '\x1b[0m',
        'bg_black': '\x1b[40m',
        'bg_red': '\x1b[41m',
        'bg_green': '\x1b[42m',
        'bg_yellow': '\x1b[43m',
        'bg_blue': '\x1b[44m',
        'bg_magenta': '\x1b[45m',
        'bg_cyan': '\x1b[46m',
        'bg_white': '\x1b[47m',
        'bg_normal': '\x1b[49m',
        'light_gray': '\x1b[0;90m',
        'light_red': '\x1b[0;91m',
        'light_green': '\x1b[0;92m',
        'light_yellow': '\x1b[0;93m',
        'light_blue': '\x1b[0;94m',
        'light_magenta': '\x1b[0;95m',
        'light_cyan': '\x1b[0;96m',
        'light_white': '\x1b[0;97m',
        'bg_light_gray': '\x1b[0;100m',
        'bg_light_red': '\x1b[0;101m',
        'bg_light_green': '\x1b[0;102m',
        'bg_light_yellow': '\x1b[0;103m',
        'bg_light_blue': '\x1b[0;104m',
        'bg_light_magenta': '\x1b[0;105m',
        'bg_light_cyan': '\x1b[0;106m',
        'bg_light_white': '\x1b[0;107m',
        'bold': '\x1b[1m',
        'underline': '\x1b[4m',
        'blink': '\x1b[5m'
    }

    if not advanced_mode:
        if colors[0] in color_code:
            print(color_code[colors[0]] + Input + '\x1b[0m', end=print_END)
        else:
            print("[print_Color] ERROR: Invalid color input!!!")
    else:
        substrings = Input.split('~*')
        if len(substrings) != len(colors) + 1:
            print("[print_Color] ERROR: Number of colors and number of '~*' don't match!!!")
        else:
            for sub_str, color in zip(substrings, ['normal'] + colors):
                if color in color_code:
                    print(color_code[color] + sub_str + '\x1b[0m', end='')
                else:
                    print(f"\n[print_Color] ERROR: Invalid color!!! The input color: '{color}' input list index: {colors.index(color)}")
            print('', end=print_END)

## data processing with image

In [ ]:
GEN_IMG_COUNT = 40000
SIF_ARG = 0
SIF_GEN = 0
SIF_TEST = 0
SIF_TEST_TVS = 0
ADG_VM = False
DDA = 3
#data_augmentation
def data_augmentation(image):
    new_image = np.zeros_like(image)
    # Resize image to a random size between 22 and 28
    new_size = np.random.randint(22, 28)
    image = zoom(image, (new_size / image.shape[0], new_size / image.shape[1], 1))
    pad_size = (np.random.randint(1, 20), np.random.randint(1, 20))
    # Choose a random side or corner to pad from
    side_or_corner = np.random.choice(['top', 'bottom', 'left', 'right', 'top-left', 'top-right', 'bottom-left', 'bottom-right', 'AD'])

    # Set the padding size based on the chosen side or corner
    if side_or_corner == 'top':
        pad_size = ((pad_size[0], 0), (0, 0), (0, 0))
    elif side_or_corner == 'bottom':
        pad_size = ((0, pad_size[0]), (0, 0), (0, 0))
    elif side_or_corner == 'left':
        pad_size = ((0, 0), (pad_size[1], 0), (0, 0))
    elif side_or_corner == 'right':
        pad_size = ((0, 0), (0, pad_size[1]), (0, 0))
    elif side_or_corner == 'top-left':
        pad_size = ((pad_size[0], 0), (pad_size[1], 0), (0, 0))
    elif side_or_corner == 'top-right':
        pad_size = ((pad_size[0], 0), (0, pad_size[1]), (0, 0))
    elif side_or_corner == 'bottom-left':
        pad_size = ((0, pad_size[0]), (pad_size[1], 0), (0, 0))
    elif side_or_corner == 'bottom-right':
        pad_size = ((0, pad_size[0]), (0, pad_size[1]), (0, 0))
    else: #'AD'
        pad_size = ((pad_size[1], pad_size[1]), (pad_size[1], pad_size[1]), (0, 0))
        
    padded_image = np.pad(image, pad_size, mode='constant')

    new_image = padded_image
    image = new_image
    
    resized_image = cv2.resize(image, (28, 28))
    image = resized_image[:, :, np.newaxis]
    new_image = image
    
    crop_size = np.random.randint(23, 28)
    # Choose a random direction
    direction = np.random.choice(['up', 'down', 'left', 'right'])

    # Set the starting position of the cropped image based on the chosen direction
    if direction == 'up':
        start_x = np.random.randint(0, image.shape[1] - crop_size)
        start_y = 0
    elif direction == 'down':
        start_x = np.random.randint(0, image.shape[1] - crop_size)
        start_y = image.shape[0] - crop_size
    elif direction == 'left':
        start_x = 0
        start_y = np.random.randint(0, image.shape[0] - crop_size)
    else:  # 'right'
        start_x = image.shape[1] - crop_size
        start_y = np.random.randint(0, image.shape[0] - crop_size)

    # Crop the image
    cropped_image = image[start_y:start_y+crop_size, start_x:start_x+crop_size]

    image = cropped_image
    new_image = image
    resized_image = cv2.resize(image, (28, 28))
    image = resized_image[:, :, np.newaxis]
    new_image = image
    for i in range(np.random.randint(0, 12)):
        image_copy = image.copy()  # Create a copy of the original image
        height, width = image_copy.shape[0], image_copy.shape[1]
        x1, y1 = np.random.randint(0, width), np.random.randint(0, height)
        x2, y2 = np.random.randint(0, width), np.random.randint(0, height)

        color = (random.uniform(0.1, 1), random.uniform(0.1, 1), random.uniform(0.1, 1))  # RGB color
        thickness = np.random.randint(1, 2)  # Random thickness

        cv2.line(image_copy, (x1, y1), (x2, y2), color, thickness)
        image = image_copy
    new_image = image
    noise_func = np.random.choice(['L1', 'L2', 'L3', 'none'])
    if noise_func == 'L3':
        intensityL2 = random.uniform(0.1, 0.15)
        intensityL1 = random.uniform(0.1, 0.15)
    else:
        intensityL2 = random.uniform(0.1, 0.25)
        intensityL1 = random.uniform(0.1, 0.25)
    if noise_func == 'L2' or noise_func == 'L3':
        for i in range(0, image.shape[0], 4):
            for j in range(0, image.shape[1], 4):
                block = image[i:i+4, j:j+4]
                block = (np.random.rand() * intensityL2 + 1) * block
                new_image[i:i+4, j:j+4] = block
        image = new_image      
    elif noise_func == 'L1' or noise_func == 'L3': 
        for i in range(0, image.shape[0], 2):
            for j in range(0, image.shape[1], 2):
                block = image[i:i+2, j:j+2]
                block = (np.random.rand() * intensityL1 + 1) * block
                new_image[i:i+2, j:j+2] = block
    return new_image
#data_augmentation_TVS (DO NOT CHANGE)
def data_augmentation_TVS(image):
    new_image = np.zeros_like(image)
    # Resize image to a random size between 22 and 28
    new_size = np.random.randint(22, 28)
    image = zoom(image, (new_size / image.shape[0], new_size / image.shape[1], 1))
    pad_size = (np.random.randint(1, 20), np.random.randint(1, 20))
    # Choose a random side or corner to pad from
    side_or_corner = np.random.choice(['top', 'bottom', 'left', 'right', 'top-left', 'top-right', 'bottom-left', 'bottom-right', 'AD'])

    # Set the padding size based on the chosen side or corner
    if side_or_corner == 'top':
        pad_size = ((pad_size[0], 0), (0, 0), (0, 0))
    elif side_or_corner == 'bottom':
        pad_size = ((0, pad_size[0]), (0, 0), (0, 0))
    elif side_or_corner == 'left':
        pad_size = ((0, 0), (pad_size[1], 0), (0, 0))
    elif side_or_corner == 'right':
        pad_size = ((0, 0), (0, pad_size[1]), (0, 0))
    elif side_or_corner == 'top-left':
        pad_size = ((pad_size[0], 0), (pad_size[1], 0), (0, 0))
    elif side_or_corner == 'top-right':
        pad_size = ((pad_size[0], 0), (0, pad_size[1]), (0, 0))
    elif side_or_corner == 'bottom-left':
        pad_size = ((0, pad_size[0]), (pad_size[1], 0), (0, 0))
    elif side_or_corner == 'bottom-right':
        pad_size = ((0, pad_size[0]), (0, pad_size[1]), (0, 0))
    else: #'AD'
        pad_size = ((pad_size[1], pad_size[1]), (pad_size[1], pad_size[1]), (0, 0))
        
    padded_image = np.pad(image, pad_size, mode='constant')

    new_image = padded_image
    image = new_image
    
    resized_image = cv2.resize(image, (28, 28))
    image = resized_image[:, :, np.newaxis]
    new_image = image
    
    crop_size = np.random.randint(23, 28)
    # Choose a random direction
    direction = np.random.choice(['up', 'down', 'left', 'right'])

    # Set the starting position of the cropped image based on the chosen direction
    if direction == 'up':
        start_x = np.random.randint(0, image.shape[1] - crop_size)
        start_y = 0
    elif direction == 'down':
        start_x = np.random.randint(0, image.shape[1] - crop_size)
        start_y = image.shape[0] - crop_size
    elif direction == 'left':
        start_x = 0
        start_y = np.random.randint(0, image.shape[0] - crop_size)
    else:  # 'right'
        start_x = image.shape[1] - crop_size
        start_y = np.random.randint(0, image.shape[0] - crop_size)

    # Crop the image
    cropped_image = image[start_y:start_y+crop_size, start_x:start_x+crop_size]

    image = cropped_image
    new_image = image
    resized_image = cv2.resize(image, (28, 28))
    image = resized_image[:, :, np.newaxis]
    new_image = image
    for i in range(np.random.randint(0, 12)):
        image_copy = image.copy()  # Create a copy of the original image
        height, width = image_copy.shape[0], image_copy.shape[1]
        x1, y1 = np.random.randint(0, width), np.random.randint(0, height)
        x2, y2 = np.random.randint(0, width), np.random.randint(0, height)

        color = (random.uniform(0.1, 1), random.uniform(0.1, 1), random.uniform(0.1, 1))  # RGB color
        thickness = np.random.randint(1, 2)  # Random thickness

        cv2.line(image_copy, (x1, y1), (x2, y2), color, thickness)
        image = image_copy
    new_image = image
    noise_func = np.random.choice(['L1', 'L2', 'L3', 'none'])
    if noise_func == 'L3':
        intensityL2 = random.uniform(0.01, 0.2)
        intensityL1 = random.uniform(0.01, 0.2)
    else:
        intensityL2 = random.uniform(0.01, 0.35)
        intensityL1 = random.uniform(0.01, 0.35)
    if noise_func == 'L2' or noise_func == 'L3':
        for i in range(0, image.shape[0], 4):
            for j in range(0, image.shape[1], 4):
                block = image[i:i+4, j:j+4]
                block = (np.random.rand() * intensityL2 + 1) * block
                new_image[i:i+4, j:j+4] = block
        image = new_image      
    elif noise_func == 'L1' or noise_func == 'L3': 
        for i in range(0, image.shape[0], 2):
            for j in range(0, image.shape[1], 2):
                block = image[i:i+2, j:j+2]
                block = (np.random.rand() * intensityL1 + 1) * block
                new_image[i:i+2, j:j+2] = block
    return new_image
#save_images_to_dir
def save_images_to_dir(images, labels, dir_path):
    # create the directory if it doesn't exist
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    # iterate over the images and labels
    for i, (image, label) in enumerate(zip(images, labels)):
        # only save every 10th image
        if i % 10 == 0:
            # get the class label
            class_label = np.argmax(label)
            # create the file path
            file_path = os.path.join(dir_path, f'image_{i}_class_{class_label}.png')
            # save the image to the file path
            plt.imsave(file_path, image.squeeze(), cmap='gray')
print('loading Dataset...')
# load dataset
with np.load('mnist.npz') as data:
    x_train, y_train = data['x_train'], data['y_train']
    x_test, y_test = data['x_test'], data['y_test']

# reshape dataset to have a single channel
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

# one hot encode target values
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#gen img
def generate_image(number, font):
    img = Image.new('L', (28, 28), color=random.randint(50, 240))
    number_color = random.randint(180, 255)
    draw = ImageDraw.Draw(img)
    
    # Drawing number
    text_length = font.getlength(str(number))
    x_offset = random.randint(-8, 8)  # Generate a random offset between -8 and 8 for x
    y_offset = random.randint(-8, 8)  # Generate a random offset between -8 and 8 for y
    x = (img.width - text_length) // 2 + x_offset
    y = (img.height - font.size) // 2 + y_offset
    draw.text((x, y), str(number), font=font, fill=number_color)
    
    # Drawing lines
    num_lines = random.randint(0, 10)
    for _ in range(num_lines):
        start = (random.randint(0, img.width-1), random.randint(0, img.height-1))
        end = (random.randint(0, img.width-1), random.randint(0, img.height-1))
        line_width = random.randint(1, 2)
        line_color = random.randint(0, 255);  # Random grayscale value
        draw.line([start, end], fill=line_color, width=line_width)
    
    return np.expand_dims(np.array(img), axis=-1)


fonts = [
    ImageFont.truetype('arial.ttf', random.randint(6, 28)),
    ImageFont.truetype('times.ttf', random.randint(6, 28)),
    ImageFont.truetype('calibri.ttf', random.randint(6, 28)),
    ImageFont.truetype('comic.ttf', random.randint(6, 28)),
    ImageFont.truetype('georgia.ttf', random.randint(6, 28)),
    ImageFont.truetype('impact.ttf', random.randint(6, 28)),
    ImageFont.truetype('tahoma.ttf', random.randint(6, 28)),
    ImageFont.truetype('verdana.ttf', random.randint(6, 28)),
]
print('Generating extra imgs...')
numbers = [randint(0, 9) for _ in range(GEN_IMG_COUNT)]
images = [generate_image(number, choice(fonts)) for number in numbers]

GEN_IMG_x_train, GEN_IMG_x_test, GEN_IMG_y_train, GEN_IMG_y_test = train_test_split(images, numbers, test_size=0.20)
GEN_IMG_y_train = to_categorical(GEN_IMG_y_train, num_classes=10)
GEN_IMG_y_test = to_categorical(GEN_IMG_y_test, num_classes=10)

x_train = np.concatenate([x_train, GEN_IMG_x_train])
y_train = np.concatenate([y_train, GEN_IMG_y_train])
x_test = np.concatenate([x_test, GEN_IMG_x_test])
y_test = np.concatenate([y_test, GEN_IMG_y_test])
# data processing
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
#summarize_datagen func
def summarize_datagen(datagen):
    print('ImageDataGenerator Conf:')
    config = datagen.__dict__
    for key, value in config.items():
        print(f">   {key}: {value}")

# create a data generator
#train
datagen = ImageDataGenerator(rotation_range=20,
                             horizontal_flip=False,
                             zoom_range = 0.15, 
                             width_shift_range=0.1, 
                             height_shift_range=0.1,
                             #brightness_range=(0.95,1.05),
                             preprocessing_function=lambda x: data_augmentation(1-x))
#TVS (DO NOT CHANGE)
datagen_TVS = ImageDataGenerator(rotation_range=20,
                                 horizontal_flip=False,
                                 zoom_range = 0.15, 
                                 width_shift_range=0.1, 
                                 height_shift_range=0.1,
                                 #brightness_range=(0.95,1.05),
                                 preprocessing_function=lambda x: data_augmentation_TVS(1-x))
if ADG_VM: 
    summarize_datagen(datagen)
    summarize_datagen(datagen_TVS)    
#GAD_TVS
print(f'Generating augmented data TVS [DDA: 3]...')
for i in range(3):
    print(f'>   Generating ADB TVS [{i}]...')
    # prepare an iterators to scale images
    test_iterator_TVS = datagen_TVS.flow(x_test, y_test, batch_size=len(x_test))

    # get augmented data
    x_test_augmented, y_test_augmented = test_iterator_TVS.next()
    
    print(f'>   \---Adding the Generated ADB TVS...')
    # append augmented data to original data
    x_test = np.concatenate([x_test, x_test_augmented])
    y_test = np.concatenate([y_test, y_test_augmented])
#GAD_train
print(f'Generating augmented data [DDA: {str(DDA)}]...')
if DDA > 0:
    for i in range(DDA):
        print(f'>   Generating ADB[{i}]...')
        # prepare an iterators to scale images
        train_iterator = datagen.flow(x_train, y_train, batch_size=len(x_train))

        # get augmented data
        x_train_augmented, y_train_augmented = train_iterator.next()
        
        print(f'>   \---Adding the Generated ADB...')
        # append augmented data to original data
        x_train = np.concatenate([x_train, x_train_augmented])
        y_train = np.concatenate([y_train, y_train_augmented])
else:
    print('>    Replacing data with augmented data...')
    # prepare an iterators to scale images
    train_iterator = datagen.flow(x_train, y_train, batch_size=len(x_train))

    # get augmented data
    x_train_augmented, y_train_augmented = train_iterator.next()
    
    # append augmented data to original data
    x_train =  x_train_augmented
    y_train =  y_train_augmented
#STS
print('Saving test samples...')
# save augmented data to directories
if SIF_ARG:
    print('>    Saving [test_TS_ARG]...')
    save_images_to_dir(x_train_augmented, y_train_augmented, 'test_TS_ARG')
if SIF_GEN:
    print('>    Saving [test_TS_GEN]...')
    save_images_to_dir(GEN_IMG_x_train, GEN_IMG_y_train, 'test_TS_GEN')
if SIF_TEST:
    print('>    Saving [test_TS]...')
    save_images_to_dir(x_train, y_train, 'test_TS')
if SIF_TEST_TVS:
    print('>    Saving [test_TS_TVS]...')
    save_images_to_dir(x_test, y_test, 'test_TS_TVS')
print('verbose: ')
print(f'>   Train len: {x_train.shape[0]:,}')
print(f'>   Test TVS len: {x_test.shape[0]:,}')
print('done.')

## Creating the model

In [ ]:
model = Sequential()
#Feature Extaction Size=28x28
model.add(Conv2D(64, (4, 4), activation='relu', padding='same', name='block1_conv1', input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(64, (4, 4), activation='relu', padding='same', name='block1_conv2'))
model.add(BatchNormalization())
model.add(Conv2D(64, (4, 4), activation='relu', padding='same', name='block1_conv3'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))
#_LFE(Large Feature Extaction) Size=14x14
model.add(Conv2D(128, (5, 5), activation='relu', padding='same', name='block2_conv1_LFE'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(128, (5, 5), activation='relu', padding='same', name='block2_conv2_LFE'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(128, (5, 5), activation='relu', padding='same', name='block2_conv3_LFE'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(128, (5, 5), activation='relu', padding='same', name='block2_conv4_LFE'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))
#Feature Extaction Size=7x7
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block4_conv1'))
model.add(BatchNormalization())
model.add(Dropout(0.35))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block4_conv2'))
model.add(BatchNormalization())
model.add(Dropout(0.35))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block4_conv3'))
model.add(BatchNormalization())
model.add(Dropout(0.35))
#Feature Extaction Size=7x7
model.add(Conv2D(512, (2, 2), activation='relu', padding='same', name='block5_conv1'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(512, (2, 2), activation='relu', padding='same', name='block5_conv2'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(512, (2, 2), activation='relu', padding='same', name='block5_conv3'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))
#Feature Extaction Size=3x3
model.add(Conv2D(512, (1, 1), activation='relu', padding='same', name='block6_conv1'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(512, (1, 1), activation='relu', padding='same', name='block6_conv2'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(512, (1, 1), activation='relu', padding='same', name='block6_conv3'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
#Flatten
model.add(Flatten(name='flatten1'))
#Feature Classifier
model.add(Dense(512, activation='relu', name='fc1', kernel_regularizer=l2(0.1)))
model.add(BatchNormalization())
model.add(Dense(512, activation='relu', name='fc2', kernel_regularizer=l2(0.1)))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax', name='predictions'))
# compile model
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#summary
model.summary()
#end
print('\nThe model was successfully created')

## loading the model

In [ ]:
try:
    model = load_model('MNIST_model.h5')
except (ImportError, IOError) as e:
    print_Color(f'failed to load the model ERROR:\n>  {e}', ['red'])
else:
    print_Color('loading model done.', ['green'])

## Training

In [ ]:
#early_stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=8, verbose=1, restore_best_weights = True)
#visualizer
log_dir = "logs/fit/" + datetime.datetime.now().strftime("y%Y_m%m_d%d-h%H_m%M_s%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, write_images=False, histogram_freq=1)
print_Color(f'Log dir: ~*{log_dir}', ['yellow'], advanced_mode=True)
print_Color('Training the model...\n', ['white'])
try:
    history = model.fit(x_train, y_train, epochs=256, batch_size=32, validation_data=(x_test, y_test), verbose='auto', callbacks=[early_stopping, tensorboard_callback])
except KeyboardInterrupt:
    print_Color('\nTraining stopped!', ['red'])
else:
    print_Color('Training done.\n', ['green'])

## Saving model weights

In [ ]:
model.save('MNIST_model.h5')

## Analyzing the results

In [ ]:
try:
    #loss
    plt.plot(history.history['loss'], label='loss')
    try:
        plt.plot(history.history['val_loss'], label='val_loss', color='orange')
    except (ValueError, NameError):
        print_Color('failed to load val_loss.', ['red'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.show()
    #acc
    plt.plot(history.history['accuracy'], label='accuracy')
    try:
        plt.plot(history.history['val_accuracy'], label='val_accuracy', color='orange')
    except (ValueError, NameError):
        print_Color('failed to load val_accuracy.', ['red'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.show()
except (ValueError, NameError):
    print_Color('failed to load model history.', ['red'])